In [6]:
import plotly.offline as pyo
import plotly_express as pyx
import plotly.graph_objects as go
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import plotly.figure_factory as ff
import numpy as np
from numpy import *
import pandas as pd

data = pd.read_csv("sample_semanai.csv")
data = data.drop_duplicates().iloc[:]
top_5 = data.groupby(['Clave platillo'])['Cantidad'].agg(
    {"Cuenta": len}).sort_values(
    "Cuenta", ascending=False).head(5).reset_index()
top_5 = top_5['Clave platillo']

c:\users\gtdav\miniconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning:

using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)




In [7]:
data2 = data[data["Acción"] != "Cancelación"]
data2.dropna(axis=0, subset=['Clave platillo'], inplace=True)
data2 = data2[['Cantidad','Clave platillo', 'idEncuesta']]
data2 = data2.dropna()

In [8]:
data2 = data2.groupby(['idEncuesta', 'Clave platillo'])['Cantidad'].sum().unstack().reset_index().fillna(0).set_index('idEncuesta')

Clave platillo,BAS1,BAS10,BAS11,BAS12,BAS13,BAS14,BAS2,BAS3,BAS4,BAS5,...,bebi004,bebi005,bebi006,bebi01,bebi02,bebi03,mcl90004,mcl90005,mcl90009,mcla9006
idEncuesta,,,,,,,,,,,,,,,,,,,,,
115787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115791,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
615569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
615571,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

data2 = data2.applymap(encode_units)
data2 = data2.loc[:, data2.columns.intersection(top_5)]
data2 = data2[(data2.T != 0).any()]
data2

Clave platillo,BAS1,BCA8,HAM12,MBCA63,MHAM30
idEncuesta,,,,,
115788,0,0,0,0,1
117530,0,1,0,0,0
117533,0,0,0,0,1
117536,0,0,0,1,0
117602,0,0,0,1,0
...,...,...,...,...,...
615009,1,0,0,0,0
615054,1,1,0,0,1
615342,0,1,0,0,0


In [60]:
datatemp1 = data2[data2[top_5] == 1].groupby(top_5[0]).sum().reset_index().reindex(sorted(data2.columns), axis=1)
for i in range(4):
    datatemp2 = data2[data2[top_5] == 1].groupby(top_5[i+1]).sum().reset_index().reindex(sorted(data2.columns), axis=1)
    datatemp1 = datatemp1.append(datatemp2)
datatemp['Clave platillo'] = top_5
datatemp1

Clave platillo,BAS1,BCA8,HAM12,MBCA63,MHAM30
0,783.0,473.0,710.0,531.0,1.0
0,1.0,142.0,316.0,123.0,783.0
0,316.0,168.0,1.0,190.0,710.0
0,123.0,124.0,190.0,1.0,531.0
0,142.0,1.0,168.0,124.0,473.0


In [61]:
fig = ff.create_dendrogram(datatemp1, orientation='left')
fig.show()

Clave platillo,BAS1,BCA8,HAM12,MBCA63,MHAM30
0,1.0,142.0,316.0,123.0,783.0
0,142.0,1.0,168.0,124.0,473.0
